# Download 1/2 degree plots

In [1]:
import ee, geemap
print(ee.__version__)
print(geemap.__version__)

0.1.249
0.8.8


In [2]:
ee.Initialize()

### Get region data and sentinel-2 data from GEE

In [4]:
# List sub-regions using GAUL polygon data
br = (ee.FeatureCollection("FAO/GAUL/2015/level1")
      .filterMetadata('ADM0_NAME', 'equals', 'Brazil')
      .aggregate_array('ADM1_NAME')
     )

print(br.getInfo())

['Alagoas', 'Bahia', 'Ceara', 'Espirito Santo', 'Maranhao', 'Minas Gerais', 'Paraiba', 'Pernambuco', 'Piaui', 'Rio De Janeiro', 'Rio Grande Do Norte', 'Sao Paulo', 'Sergipe', 'Name Unknown', 'Name Unknown', 'Name Unknown', 'Acre', 'Amapa', 'Amazonas', 'Distrito Federal', 'Goias', 'Mato Grosso', 'Mato Grosso Do Sul', 'Para', 'Parana', 'Rio Grande Do Sul', 'Rondonia', 'Roraima', 'Santa Catarina', 'Tocantins', 'Name Unknown']


In [5]:
# Get a sub-region from above to test grid function later
br = (ee.FeatureCollection("FAO/GAUL/2015/level1")
      .filterMetadata('ADM0_NAME', 'equals', 'Brazil')
      .filterMetadata('ADM1_NAME', 'equals', 'Amazonas')
     )

## Create a grid

### Method 1 (failed)
This method will could probably work better than the second, but I couldn't get it perfectly translated from javascript.

In [9]:
# Method 1 -- Failed, ends up in the ocean and invisible
# https://code.earthengine.google.com/cf1d2e6e8c6375d33c48f0a121293bb0#
import math

lon_start = -60.00
lon_end = -60.50
lat_start = -2.35
lat_end = -2.85

num_cells = 4
lon_edge = (lon_end-lon_start)/ math.sqrt(num_cells)
lat_edge = (lat_end-lat_start)/ math.sqrt(num_cells)

polys = []
cell_id = 0

lon = lon_start
while lon < lon_end:
    x1 = lon
    x2 = lon + lon_edge
    lon += lon_edge
    
    lat = lat_start
    while lat < lat_end:
        cell_id = cell_id + 1
        y1 = lat
        y2 = lat + lat_edge
        lat += lat_edge
        
        ee.List(polys).add(ee.Feature(ee.Geometry.Rectangle(x1, y1, x2, y2), {'label': cell_id}))

grid = ee.FeatureCollection(polys)

### Method 2
Not 100% perfect. You have to have a reasonably-sized roi drawn. If it's too big, it's hard to predict where the grid will be drawn. But hey, at least it gives me some squares.

In [24]:
# Method 2: geemap.show_youtube('N7rK2aV1R4c')
# Create a map and draw a square interactively
Map = geemap.Map()
Map.addLayer(br)
Map.center_object(br, zoom=5)

Map

Map(center=[-4.186394628415986, -64.69261254089228], controls=(WidgetControl(options=['position', 'transparent…

In [25]:
# Set name
my_roi = br

In [26]:
# Save feature
my_roi = ee.FeatureCollection(my_roi)

In [27]:
# Create grid
# https://developers.google.com/earth-engine/tutorials/community/drawing-tools

def make_grid(region, a_scale):
    """
    Creates a grid around a specified ROI.
    User inputs their reasonably small ROI.
    User inputs a scale where 100000 = 100km.
    """
    # Creates image with 2 bands ('longitude', 'latitude') in degrees
    lonLat = ee.Image.pixelLonLat()

    # Select bands, multiply times big number, and truncate
    lonGrid = (lonLat
               .select('latitude')
               .multiply(10000000)
               .toInt()
              )
    latGrid = (lonLat
              .select('longitude')
              .multiply(10000000)
              .toInt()
              )

    # Multiply lat and lon images and reduce to vectors
    grid = (lonGrid
            .multiply(latGrid)
            .reduceToVectors(
                geometry = region,
                scale = a_scale, # 100km-sized boxes needs 100,000
                geometryType = 'polygon')
           )
    
    return(grid)

In [28]:
# Make test grid and add to map
grid_55km = make_grid(br, 55000)

## Visualize selected grid tiles + images

In [29]:
# Access coordinates of grid squares
grid_dict = grid_55km.getInfo()

feats = grid_dict['features']
coord_list = []
for d in feats:
    geom = d['geometry']
    coords = geom['coordinates']
    coord_list.append(coords)

In [30]:
# Create a list of several ee.Geometry.Polygons
polys = []
for coord in coord_list:
    poly = ee.Geometry.Polygon(coord)
    polys.append(poly)

In [31]:
# Limit grid tiles to export
idx = list(range(0,100))
polys = [ polys[i] for i in idx]

In [34]:
# Make the whole grid a feature collection for export purposes
grid = ee.FeatureCollection(polys)

In [35]:
# Visualize a polygon or two
Map = geemap.Map()

Map.addLayer(grid)
Map.center_object(polys[0], zoom=5)

Map

Map(center=[-2.223337188443923, -56.57140501742681], controls=(WidgetControl(options=['position', 'transparent…

## Export grid as shapfile

In [37]:
# Set all ee.Geometry.Polygon to ee.Feature
feat_list = []
for poly in polys:
    feat = ee.Feature(poly)
    feat_list.append(feat)
print(len(feat_list))

100


In [38]:
# Export entire grid as shapefile
geemap.ee_to_shp(grid, filename='/data/6ru/amazonas_grid.shp')

Generating URL ...
Please wait ...
Data downloaded to /data/6ru/amazonas_grid.shp
